<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/190_stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Стекинг (Stacking): с CatBoost и другими моделями


## 🟢 Зеленый уровень — Базовое понимание

**Что такое стекинг?**  
Стекинг (*stacking*) — это метод ансамблирования, когда мы комбинируем несколько разных моделей (*базовых моделей*) с помощью ещё одной модели (*мета-модели*), которая учится на их предсказаниях.

**Пример с жизнью:**  
Представь, что у тебя есть три друга: программист, математик и аналитик. Каждый даёт свой прогноз, а ты нанимаешь «супер-друга», который учится правильно взвешивать их ответы, чтобы получить самый точный итог.

**Схема:**
1. Обучаем несколько **разных** моделей на исходных данных.
2. Получаем их предсказания.
3. Используем эти предсказания как новые признаки для **мета-модели**.
4. Мета-модель делает финальный прогноз.

**Почему это круто:**
- Стекинг объединяет сильные стороны разных алгоритмов.
- Может дать прирост качества, даже если Voting не помог.


In [ ]:
!pip install catboost
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

cat = CatBoostClassifier(verbose=0, random_state=42)
rf = RandomForestClassifier(random_state=42)
lr = LogisticRegression()

stack = StackingClassifier(
    estimators=[('cat', cat), ('rf', rf)],
    final_estimator=lr
)

stack.fit(X_train, y_train)
stack.score(X_test, y_test)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.7 MB/s eta 0:00:00



**🟢 Упражнения:**
1. Добавь в стек модель `KNeighborsClassifier`.
2. Замени мета-модель на `CatBoostClassifier`.
3. Посмотри, как изменится точность, если убрать CatBoost из базовых моделей.



## 🟡 Жёлтый уровень — Чуть глубже

**Почему просто Voting иногда хуже Стекинга:**  
- Voting — это фиксированное правило (среднее или большинство голосов).
- Стекинг обучает мета-модель, которая может «понять», что одной модели стоит доверять больше в одних случаях, а другой — в других.

**Как устроен StackingClassifier в sklearn:**
- **Параметр `passthrough=True`** — добавляет исходные признаки к входу мета-модели.
- **cv** — число фолдов для кросс-валидации при обучении базовых моделей (чтобы избежать утечек данных).


In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

stack2 = StackingClassifier(
    estimators=[('cat', cat), ('xgb', xgb), ('rf', rf)],
    final_estimator=LogisticRegression(),
    passthrough=True,
    cv=5
)

stack2.fit(X_train, y_train)
stack2.score(X_test, y_test)



**🟡 Упражнения:**
1. Сравни точность стека с `passthrough=True` и `passthrough=False`.
2. Замени мета-модель на `XGBClassifier`.
3. Подбери `cv` и `n_estimators` для улучшения качества.



## 🔴 Красный уровень — Для любопытных и амбициозных

**Тонкости стекинга:**
- **Data leakage** — нельзя обучать мета-модель на тех же данных, на которых базовые модели предсказывают.
- Чтобы избежать утечки, используют *out-of-fold* предсказания.
- CatBoost умеет работать со стекингом и категориальными признаками без one-hot кодирования.

**Стекинг в реальном ML-пайплайне:**
- Можно использовать стекинг как финальный шаг после отбора признаков.
- Для бустингов (CatBoost, XGBoost, LightGBM) иногда выгодно ставить их в качестве **мета-модели**, а не только базовых.


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

base_models = [CatBoostClassifier(verbose=0, random_state=42),
               XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
               RandomForestClassifier(random_state=42)]

meta_model = LogisticRegression()

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

meta_features = np.zeros((X_train.shape[0], len(base_models)))
test_meta_features = np.zeros((X_test.shape[0], len(base_models)))

for i, model in enumerate(base_models):
    test_preds_fold = []
    for train_idx, val_idx in kf.split(X_train, y_train):
        model.fit(X_train[train_idx], y_train[train_idx])
        meta_features[val_idx, i] = model.predict_proba(X_train[val_idx])[:, 1]
        test_preds_fold.append(model.predict_proba(X_test)[:, 1])
    test_meta_features[:, i] = np.mean(test_preds_fold, axis=0)

meta_model.fit(meta_features, y_train)
meta_model.score(test_meta_features, y_test)



**🔴 Упражнения:**
1. Реализуй стекинг с CatBoost в качестве мета-модели.
2. Добавь `LightGBM` в базовые модели.
3. Попробуй сделать стекинг для задачи регрессии.
